# View the notebook better [here](https://nbviewer.jupyter.org/github/anfleury/radio-project-tbd/blob/master/notebooks/route_planning.ipynb).

In [1]:
import folium
import osmnx as ox
import networkx as nx
from IPython.display import HTML, display

In C:\test\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\test\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\test\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\test\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\test\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be

In [2]:
my_map4 = folium.Map(location = [28.5011226, 77.4099794], 
                                        zoom_start = 12) 
  
folium.Marker([28.704059, 77.102490], 
              popup = 'Delhi').add_to(my_map4) 
  
folium.Marker([28.5011226, 77.4099794], 
              popup = 'GeeksforGeeks').add_to(my_map4)

# CircleMarker with radius 
folium.CircleMarker(location = [28.5011226, 77.4099794], 
                    radius = 50, popup = ' FRI ').add_to(my_map4)
  
# Add a line to the map by using line method . 
# it connect both coordiates by the line 
# line_opacity implies intensity of the line 
  
folium.PolyLine(locations = [(28.704059, 77.102490), (28.5011226, 77.4099794)], 
                line_opacity = 0.5).add_to(my_map4)

display(my_map4)

Here the route can be generated by a polyline given a list of node coordinates. 

In [3]:
# coordinates = [
#     [42.3581, -71.0636],
#     [42.82995815, -74.78991444],
#     [43.17929819, -78.56603306],
#     [43.40320216, -82.37774519],
#     [43.49975489, -86.20965845],
#     [41.4338549, -108.74485069],
#     [40.67471747, -112.29609954],
#     [39.8093434, -115.76190821],
#     [38.84352776, -119.13665678],
#     [37.7833, -122.4167]]

# # Create the map and add the line
# m = folium.Map(location=[41.9, -97.3], zoom_start=4)
# my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
# m.add_children(my_PolyLine)
# display(m)

Here is the code for plotting the map as fig and ax syntax.

In [4]:
# G = ox.graph_from_place('Manhattan, New York, USA', network_type='drive')
# fig, ax = ox.plot_graph(G)

## Getting graph and drive route
Running `graph_from_place` seems necessary, and it takes around 15 seconds to load just Manhattan Island. Suggestion here is to load this map as early as possible.

In [5]:
from time import time

start = time()
ox.config(log_console=True, use_cache=True)

G_drive = ox.graph_from_place('Manhattan Island, New York City, New York, USA',
                             network_type='drive')
startx = time()
print('Time to load graph: %.2f s.' %(startx-start))
orig_node = ox.get_nearest_node(G_drive,
                                (40.7883018, -73.951163))

dest_node = ox.get_nearest_node(G_drive,
                                (40.706018, -74.018038))
start2 = time()
print('Time to nodes: %.2f s.' %(start2-startx))
route = nx.shortest_path(G_drive,
                         orig_node,
                         dest_node,
                         weight='length')
start3 = time()
print('Time to shortest path: %.2f s.' %(start3-start2))
route_map = ox.plot_route_folium(G_drive, route)
print('Time to build map: %.2f s.' %(time()-start3))

Time to load graph: 8.93 s.
Time to nodes: 0.02 s.
Time to shortest path: 0.03 s.
Time to build map: 0.53 s.


In [6]:
folium.Marker(location=[40.7883018, -73.951163], 
        icon=folium.Icon(icon='play', color='green')
    ).add_to(route_map)

folium.Marker(location=[40.706018, -74.018038],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(route_map)

display(route_map)

### Note
Playing with `graph_from_point`. Default distance for the `dist` parameter is 1000m. Using the `start_point` takes about 4 seconds. However, only the partial route (whatever is within the 1km radius) is rendered.

Changing the distance to 10km takes about 76 seconds. 

In [7]:
start_point = (40.7883018, -73.951163)
end_point = (40.706018, -74.018038)

start = time()
ox.config(log_console=True, use_cache=True)

G_drive = ox.graph_from_point(center_point=start_point, 
                              dist=10000, #in meters
                              network_type='drive')

startx = time()
print('Time to load graph: %.2f s.' %(startx-start))
orig_node = ox.get_nearest_node(G_drive,
                                start_point)

dest_node = ox.get_nearest_node(G_drive,
                                end_point)
start2 = time()
print('Time to nodes: %.2f s.' %(start2-startx))
route = nx.shortest_path(G_drive,
                         orig_node,
                         dest_node,
                         weight='length')
start3 = time()
print('Time to shortest path: %.2f s.' %(start3-start2))
route_map = ox.plot_route_folium(G_drive, route)
print('Time to build map: %.2f s.' %(time()-start3))

Time to load graph: 45.70 s.
Time to nodes: 0.09 s.
Time to shortest path: 0.11 s.
Time to build map: 2.67 s.


In [8]:
folium.Marker(location=start_point, 
        icon=folium.Icon(icon='play', color='green')
    ).add_to(route_map)

folium.Marker(location=end_point,
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(route_map)

folium.TileLayer('cartodbdark_matter').add_to(route_map)
route_map.fit_bounds([end_point,start_point]) #SW, NE
display(route_map)

Next step here is to write modular functions and be able to tell how much time they need.

In [9]:
def load_map(orig):
    '''
    orig: coordinates, tuples.
    '''
    if type(orig)==list: #OSMNX uses tuples of coordinates
        orig = tuple(orig)
    ox.config(log_console=True, use_cache=True)
    g = ox.graph_from_point(center_point=start_point, # using from_point for now. 
                              dist=10000, #in meters
                              network_type='drive')
    return g

def get_route(orig, dest, g):
    '''
    orig, dest: LAT LONG coordinates, tuples/lists.
    g: input graph.
    '''
    if type(orig)==list or type(dest)==list: #OSMNX uses tuples of coordinates
        orig = tuple(orig)
        dest = tuple(dest)
    
    startx = time()
    orig_node = ox.get_nearest_node(g, orig)
    dest_node = ox.get_nearest_node(g, dest)
    start2 = time()
    print('Time to nodes: %.2f s.' %(start2-startx))
    
    route = nx.shortest_path(g, orig_node, dest_node, weight='length')
    start3 = time()
    print('Time to shortest path: %.2f s.' %(start3-start2))
    route_map = ox.plot_route_folium(g, route)
    print('Time to build map: %.2f s.' %(time()-start3))
    
    folium.Marker(location=orig, 
                  icon=folium.Icon(icon='play', color='green')
                 ).add_to(route_map)
    folium.Marker(location=dest,
                  icon=folium.Icon(icon='stop', color='red')
                 ).add_to(route_map)

    folium.TileLayer('cartodbdark_matter').add_to(route_map)
    route_map.fit_bounds([dest, orig]) #SW, NE
    display(route_map)
    return route_map

In [10]:
start_point = (40.7883018, -73.951163)
end_point = (40.706018, -74.018038)

In [11]:
%%time
g = load_map(start_point)

Wall time: 41 s


In [12]:
%%time
route_map = get_route(start_point, end_point, g)

Time to nodes: 0.13 s.
Time to shortest path: 0.12 s.
Time to build map: 2.88 s.


Wall time: 3.45 s


## Takeaways

1. Issue with creating graph: Small distance would cut away the route. Large distance takes a long time to load. **Suggestions**: a) calculate distance from the two points, find the mid-point, and use that to render the graph. b) Add a loading graphic / status bar for the user.
2. We can find the intersection of the 'bubbles' of the radio stations and the route, use that to generate the list of radio stations that are available.